# **Programming Challenge in Machine Learning:**
### *Author*: Ait lahmouch Nadir


In [ ]:
from google.colab import files
uploaded = files.upload()

## **Imports**

In [445]:

import tensorflow as tf  
import numpy as np
from keras.backend import sigmoid
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV



# **Extracting the data**

In [446]:
train = pd.read_csv('TrainOnMe.csv')
test = pd.read_csv('EvaluateOnMe.csv')

train.pop('id')
test.pop(test.columns[0])

train = train.dropna()
test = test.dropna()

train = train.drop(train[train.values  == '?'].index)
test = test.drop(test[test.values  == '?'].index)

# **Preprocessing using OneHotEncoder**


In [447]:
train_onehot = pd.get_dummies(train, columns = ['x5', 'x6'])
test_onehot = pd.get_dummies(test, columns = ['x5', 'x6'])

# **Splitting and transforming the data**

In [448]:
# x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.33)
# X_train = np.asarray(x_train).astype('float64')
# X_test = np.asarray(x_test).astype('float64')

X_train = train_onehot
y_train = X_train.pop('y')
X_test = test_onehot


## **Random forest classfier**

In [449]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=150)

rfc.fit(X_train, y_train)

y_pred_rf = rfc.predict(X_test)
# print("Accuracy: ", (y_test == y_pred_rf).sum()/len(y_test))

### **Scaling the data to use in neural networks**

In [450]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

#Processing the data
labele = preprocessing.LabelEncoder()
labele.fit(["Atsuto", "Bob", "Jörg"])
y_train_NN = labele.transform(y_train)
y_test_NN = labele.transform(y_test)

#scaling
normalized = scaler.fit_transform(np.asarray(X_train).astype('float64'))
X_train_NN = scaler.inverse_transform(normalized)
normalized_t = scaler.fit_transform(np.asarray(X_test).astype('float64'))
X_test_NN = scaler.inverse_transform(normalized_t)

## **Model ANN 1**

In [ ]:
ANN_1 = tf.keras.models.Sequential()  
ANN_1.add(tf.keras.layers.Dense(1000, activation= "relu"))
ANN_1.add(tf.keras.layers.Dense(10, activation= "softmax"))  

ANN_1.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])  

history = ANN_1.fit(X_train_NN, y_train_NN, epochs=200)  

# val_loss, acc = ANN_1.evaluate(X_test_NN, y_test_NN)  
# print(acc)

## **Model ANN 2**

In [ ]:
ANN_2 = tf.keras.models.Sequential()  

ANN_2.add(tf.keras.layers.Dense(1000, activation= "tanh"))
ANN_2.add(tf.keras.layers.Dense(10, activation= "softmax")) 

ANN_2.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy']) 

history = ANN_2.fit(X_train_NN, y_train_NN, epochs=120) 
#val_loss, acc_2 = ANN_2.evaluate(X_test_NN, y_test_NN)
#print(acc_2)

In [453]:
probas_1 = ANN_1.predict(X_test_NN)
probas_2 = ANN_2.predict(X_test_NN)

labels_1 = np.argmax(probas_1, axis=-1) 
labels_2 = np.argmax(probas_2, axis=-1) 

y_pred_NN_1 = []
y_pred_NN_2 = []

for e in labels_1:
  if e == 0:
    y_pred_NN_1.append('Atsuto')
  elif e == 1:
    y_pred_NN_1.append('Bob')
  else:
    y_pred_NN_1.append('Jörg')

for e in labels_2:
  if e == 0:
    y_pred_NN_2.append('Atsuto')
  elif e == 1:
    y_pred_NN_2.append('Bob')
  else:
    y_pred_NN_2.append('Jörg')

## **Gradient boost**

In [454]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(n_estimators= 1000)
gbc.fit(X_train, y_train)
y_pred_gb = gbc.predict(X_test)

#print(sum(y_pred_gb == y_test)/len(y_test))

## **Xgboost**

In [455]:

xgbc = XGBClassifier()

xgbc.fit(np.asarray(X_train).astype('float64'), y_train)

y_pred_x = xgbc.predict(np.asarray(X_test).astype('float64'))

#print("Accuracy: ", (y_test == y_pred_x).sum()/len(y_test))



## **Cross validation**

In [456]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(rfc, X_train, y_train, cv=10)
print("Random Forest = ", sum(scores/len(scores)))

scores = cross_val_score(gbc, X_train, y_train, cv=10)
print("gradient = ", sum(scores/len(scores)))

scores = cross_val_score(xgbc, np.asarray(X_train).astype('float64'), y_train, cv=10)
print("xgboost = ", sum(scores/len(scores)))



Random Forest =  0.8512828282828282
gradient =  0.8713434343434342
xgboost =  0.8753838383838383


## **PREDICTING**

In my last model, i will use votingclassifier. And i will use as estimators, the gradientboosted one, the randomforest and the xgboost.

In [457]:
from sklearn.ensemble import VotingClassifier 

estimator = []
estimator.append(('XGB', xgbc))
estimator.append(('RF', rfc))
estimator.append(('GB', gbc))

vot_soft = VotingClassifier(estimators = estimator, voting ='soft') 
vot_soft.fit(np.asarray(X_train).astype('float64'), y_train) 
y_pred = vot_soft.predict(np.asarray(X_test).astype('float64')) 

In [458]:
scores = cross_val_score(vot_soft, np.asarray(X_train).astype('float64'), y_train, cv=10)
print("voting = ", sum(scores/len(scores)))

voting =  0.8784343434343433


In [459]:
print("Res xgb: ", (y_pred == y_pred_x).sum()/len(y_pred))
print("Res gb: ", (y_pred == y_pred_gb).sum()/len(y_pred))
print("Res rf: ", (y_pred == y_pred_rf).sum()/len(y_pred))

Res xgb:  0.9565
Res gb:  0.9767
Res rf:  0.9326


In [460]:
labels = open("105985.txt", "w")
for line in y_pred:
  labels.write(line)
  labels.write("\n")
labels.close()

In [461]:
files.download('105985.txt') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>